In [1]:
# !pip install selenium beautifulsoup4 webdriver-manager

In [2]:
# !pip install gradio

In [1]:
from bs4 import BeautifulSoup
import requests
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
import ollama
import bs4
import gradio as gr
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import OllamaEmbeddings
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
import time
from webdriver_manager.chrome import ChromeDriverManager
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException

g:\NBE\EBook\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
USER_AGENT environment variable not set, consider setting it to identify your requests.


In [2]:
# Selenium setup for Chrome
def setup_selenium_driver():
    chrome_options = Options()
    chrome_options.add_argument("--headless")  # Run in headless mode
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")

    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=chrome_options)
    return driver

def extract_content_from_url(url):
    driver = setup_selenium_driver()
    driver.get(url)

    # Let the page load completely (adjust if necessary)
    time.sleep(5)  # Adjust sleep time as needed for content to load

    # Get the page source after JavaScript execution
    page_source = driver.page_source
    driver.quit()

    # Parse with BeautifulSoup
    soup = BeautifulSoup(page_source, 'html.parser')

    # Extract title
    title = soup.title.string if soup.title else 'No Title'

    # Extract all sections and subsections
    content = []
    for header in soup.find_all(['h1', 'h2', 'h3', 'h4', 'h5', 'h6']):
        section_title = header.get_text().strip()
        section_content = []

        # Get the content under each section
        for sibling in header.find_next_siblings():
            if sibling.name and sibling.name.startswith('h'):
                break
            section_content.append(sibling.get_text().strip())

        content.append(f"{section_title}\n{''.join(section_content)}")

    # Join all sections and subsections together
    full_content = "\n\n".join(content)

    # Return the content wrapped in a Document object 
    return {"title": title, "content": full_content}

# URL to scrape
url = "https://www.nbe.com.eg/NBE/E/#/AR/Home"

# Scrape and parse the content
document = extract_content_from_url(url)

# Output the result 
print(f"Title: {document['title']}")
print(f"Content: {document['content']}")


Title: البنك الأهلى المصرى - الصفحة الرئيسية
Content: الحسابات
حسابات جاريةحسابات التوفيرافتح حسابك وإنت في مكانكالمزيد

شهادات الادخار
عملة محليةعملة أجنبيةشهادات بلاديشهادات الاستثمارالمزيد

البطاقات
بطاقات الإئتمانبطاقات الخصم المباشرالبطاقات المدفوعة مقدمامحددات البطاقاتمحددات البطاقاتكيفية تفعيل البطاقاتالمزيد

القروض
القرض الشخصي النقديقرض السيارةمبادرة البنك المركزي المصري - التمويل العقاريالمزيد

الودائع
الودائع - عملة محليةالودائع - عملات أجنبيةالمزيد

الخدمات
الأوراق الماليةأمناء الأستثمارالتحويلات الخارجيةالتأمين البنكيالمزيد

الخدمات الالكترونية
الاهلي نت – افرادالاهلى موبايلالأهلي فون كاشفروع الخدمة الالكترونيةخدمة الأهلي للتسوق عبر الإنترنتالفرع المتنقلالأهلي واتسابخدمة المساعد الآلي التفاعلي NBE Chatbot

خدمات الحسابات
خدمة الكشف الإلكتروني للحساباتخدمة ال sweeping

الحسابات  و البطاقات
حسابات جاريهبطاقات الشركاتالمزيد

شهادات الادخار
عملة محليةعملة أجنبيةالمزيد

تمويل الشركات
تمويل الشركات والقروض المشتركةالمزيد

خدمات المرتبات
برامج الاهلي للمرتباتخدمة الراتب المقدمالم

In [21]:
# import json
# filename = "extracted_document.txt"

# # Save the dictionary as a JSON string in the text file
# with open(filename, 'w', encoding='utf-8') as file:
#     json.dump(document, file, ensure_ascii=False, indent=4)

# print(f"Document saved as {filename}")

Document saved as extracted_document.txt


In [12]:
# import json

# # Define the filename
# filename = "extracted_document.txt"  # Update with the actual path

# # Open the file and read the contents
# with open(filename, 'r', encoding='utf-8') as file:
#     document = json.load(file)



In [3]:


document

{'title': 'البنك الأهلى المصرى - الصفحة الرئيسية',
 'content': 'الحسابات\nحسابات جاريةحسابات التوفيرافتح حسابك وإنت في مكانكالمزيد\n\nشهادات الادخار\nعملة محليةعملة أجنبيةشهادات بلاديشهادات الاستثمارالمزيد\n\nالبطاقات\nبطاقات الإئتمانبطاقات الخصم المباشرالبطاقات المدفوعة مقدمامحددات البطاقاتمحددات البطاقاتكيفية تفعيل البطاقاتالمزيد\n\nالقروض\nالقرض الشخصي النقديقرض السيارةمبادرة البنك المركزي المصري - التمويل العقاريالمزيد\n\nالودائع\nالودائع - عملة محليةالودائع - عملات أجنبيةالمزيد\n\nالخدمات\nالأوراق الماليةأمناء الأستثمارالتحويلات الخارجيةالتأمين البنكيالمزيد\n\nالخدمات الالكترونية\nالاهلي نت – افرادالاهلى موبايلالأهلي فون كاشفروع الخدمة الالكترونيةخدمة الأهلي للتسوق عبر الإنترنتالفرع المتنقلالأهلي واتسابخدمة المساعد الآلي التفاعلي NBE Chatbot\n\nخدمات الحسابات\nخدمة الكشف الإلكتروني للحساباتخدمة ال sweeping\n\nالحسابات \xa0و البطاقات\nحسابات جاريهبطاقات الشركاتالمزيد\n\nشهادات الادخار\nعملة محليةعملة أجنبيةالمزيد\n\nتمويل الشركات\nتمويل الشركات والقروض المشتركةالمزيد\n\nخدمات المرت

In [4]:


splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,  # Define the size of each chunk
    chunk_overlap=100  # Define overlap between chunks
)



# Split the document's content into chunks
chunks = splitter.split_text(document['content'])


# Output the chunks
for i, chunk in enumerate(chunks):
    print(f"Chunk {i+1}:")
    print(chunk)
    print("-" * 80)

Chunk 1:
الحسابات
حسابات جاريةحسابات التوفيرافتح حسابك وإنت في مكانكالمزيد

شهادات الادخار
عملة محليةعملة أجنبيةشهادات بلاديشهادات الاستثمارالمزيد

البطاقات
بطاقات الإئتمانبطاقات الخصم المباشرالبطاقات المدفوعة مقدمامحددات البطاقاتمحددات البطاقاتكيفية تفعيل البطاقاتالمزيد

القروض
القرض الشخصي النقديقرض السيارةمبادرة البنك المركزي المصري - التمويل العقاريالمزيد

الودائع
الودائع - عملة محليةالودائع - عملات أجنبيةالمزيد

الخدمات
الأوراق الماليةأمناء الأستثمارالتحويلات الخارجيةالتأمين البنكيالمزيد

الخدمات الالكترونية
الاهلي نت – افرادالاهلى موبايلالأهلي فون كاشفروع الخدمة الالكترونيةخدمة الأهلي للتسوق عبر الإنترنتالفرع المتنقلالأهلي واتسابخدمة المساعد الآلي التفاعلي NBE Chatbot

خدمات الحسابات
خدمة الكشف الإلكتروني للحساباتخدمة ال sweeping

الحسابات  و البطاقات
حسابات جاريهبطاقات الشركاتالمزيد

شهادات الادخار
عملة محليةعملة أجنبيةالمزيد

تمويل الشركات
تمويل الشركات والقروض المشتركةالمزيد

خدمات المرتبات
برامج الاهلي للمرتباتخدمة الراتب المقدمالمزيد
-----------------------------------------

In [5]:
from sentence_transformers import SentenceTransformer
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.schema import Document

documents = [Document(page_content=chunk) for chunk in chunks]

# Load the embedding model from the SentenceTransformers library
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')
# embedding_model = SentenceTransformer('danfeg/AraBERT_Finetuned-COMB-7443')
# Use HuggingFaceEmbeddings to wrap the model for compatibility with LangChain
embeddings = HuggingFaceEmbeddings(model_name='all-MiniLM-L6-v2')
# embeddings = HuggingFaceEmbeddings(model_name='danfeg/AraBERT_Finetuned-COMB-7443')

vectordb = FAISS.from_documents(documents, embeddings)

retriever = vectordb.as_retriever()



g:\NBE\EBook\venv\Lib\site-packages\huggingface_hub\file_download.py:159: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\M\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
g:\NBE\EBook\venv\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_token

In [6]:
from langchain_groq import ChatGroq
groq_api_key="key"
llm=ChatGroq(groq_api_key=groq_api_key,
         model_name="mixtral-8x7b-32768")

from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_template("""
Answer the question in Arabic using only the provided context.
Provide a clear and concise answer, and ensure it addresses the question accurately.
search about the words in data and retrieve it
                                          
<context>
{context}
</context>

Question: {input}
""")


In [28]:
def is_question(sentence):
    question_words = ["متى","اين","؟","?","امتي","ازاي","فين","ما","ماذا", "كيف", "لماذا", "أين", "من", "هل"]

    for word in question_words:
        if word in sentence:
            return True
    else:    
      return False         

In [30]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langdetect import detect


def get_response(question):
    

   language = detect(question)
   qu=is_question(question)
   if language == 'ar' and qu==True:
        document_chain=create_stuff_documents_chain(llm,prompt)
        retrieval_chain=create_retrieval_chain(retriever,document_chain)    
        response=retrieval_chain.invoke({"input":question})
        return response["answer"]
   else: 
       return None


In [32]:
# question="اين يوجد البنك؟"
# question="what is NBE"
# question="ما هي انواع الحسابات؟"
question="صباح الخير"
ans=get_response(question)
print(ans)


None
